In [1]:
import findspark
findspark.init()


In [5]:
from pyspark.sql import SparkSession
import pyspark
from pyspark.context import SparkContext
conf = pyspark.SparkConf()
# conf.set('spark.app.name', app_name) # Optional configurations
import re
import pandas as pd




In [3]:
from pyspark.sql.context import SQLContext
from pyspark.sql.session import SparkSession
sc = pyspark.SparkContext.getOrCreate(conf=conf)


sqlContext = SQLContext(sc)
spark = SparkSession(sc)

In [7]:
import glob
raw_data_files=glob.glob('*.log')
raw_data_files

['messages2.log']

In [8]:
base_df=spark.read.text(raw_data_files)
base_df.printSchema()

root
 |-- value: string (nullable = true)



In [10]:
from pyspark.sql.functions import split, regexp_extract
split_df = base_df.select(regexp_extract('value', r'\d{2}\-\d{2}\-\d{4}', 0).alias('Date'),
                          regexp_extract('value', r'\d{2}\:\d{2}\:\d{2}', 0).alias('Time'),
                          regexp_extract('value', r'\d{12}', 0).alias('terIp'),
                          regexp_extract('value', r'test', 0).alias('Vendor'),
                          regexp_extract('value', r'[A-Z]{3} [0-9]{3}[A-Z]{1}|[A-Z]{4}[0-9]{3}[A-Z]{1}', 0).alias('cph'),
                          regexp_extract('value', r'[0-9]KM/H', 0).alias('Velocity'),
                          regexp_extract('value', r'[0-9]\.[0-9]+\,E', 0).alias('Longitude'),
                          regexp_extract('value', r'[0-9]\.[0-9]+\,N', 0).alias('Latitude'),
                          regexp_extract('value', r'"poweralarm":"1"|"poweralarm":"0"', 0).alias('PowerAlarm'),
                         
                    
                          regexp_extract('value',r'"speedDisConn":"1"|"speedDisConn":"0"' , 0).alias('SpeedDisconn'))
split_df.show(truncate=False)


+----------+--------+------------+------+--------+--------+---------+---------+----------------+------------------+
|Date      |Time    |terIp       |Vendor|cph     |Velocity|Longitude|Latitude |PowerAlarm      |SpeedDisconn      |
+----------+--------+------------+------+--------+--------+---------+---------+----------------+------------------+
|05-05-2020|11:47:28|016100005886|test  |KBJ 473H|0KM/H   |6.80370,E|0.82550,N|"poweralarm":"1"|"speedDisConn":"1"|
|05-05-2020|11:47:32|016100002106|test  |KCG 550E|0KM/H   |7.64278,E|0.05730,N|"poweralarm":"0"|"speedDisConn":"1"|
|05-05-2020|11:47:35|016100000628|test  |KBT 283L|0KM/H   |6.91717,E|0.64543,N|"poweralarm":"0"|"speedDisConn":"0"|
|05-05-2020|11:47:37|016100000988|test  |KCS 087S|0KM/H   |4.75194,E|1.33600,N|"poweralarm":"0"|"speedDisConn":"1"|
|05-05-2020|11:47:41|016100000045|test  |KCJ 245A|7KM/H   |6.64362,E|0.56697,N|"poweralarm":"0"|"speedDisConn":"1"|
|05-05-2020|11:47:44|016100000455|test  |KBM 861H|4KM/H   |8.47730,E|0.8

In [11]:
split_df.toPandas().to_csv('mycsv.csv')